### Imports

In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [50]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer

[nltk_data] Downloading package punkt to /home/ananthan2k/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ananthan2k/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/ananthan2k/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Read from file

In [51]:
files = os.listdir('./data')
files.sort()
print(files)

['001.txt', '002.txt', '003.txt', '004.txt', '005.txt', '006.txt', '007.txt', '008.txt', '009.txt', '010.txt', '011.txt', '012.txt', '013.txt', '014.txt', '015.txt', '016.txt', '017.txt', '018.txt', '019.txt', '020.txt', '021.txt', '022.txt', '023.txt', '024.txt', '025.txt', '026.txt', '027.txt', '028.txt', '029.txt', '030.txt', '031.txt', '032.txt', '033.txt', '034.txt', '035.txt', '036.txt', '037.txt', '038.txt', '039.txt', '040.txt', '041.txt', '042.txt', '043.txt', '044.txt', '045.txt', '046.txt', '047.txt', '048.txt', '049.txt', '050.txt']


In [58]:
# store each of the text in an array
doc = []

for file in files:
    with open('./data/{}'.format(file)) as fd:
        # each line in the file is read
        lines = fd.readlines()
        # each line in the file is append to doc_lines
        doc_lines = ""
        for line in lines:
            doc_lines+=line
        doc.append(doc_lines)
            

In [75]:
docs = np.array(doc)

In [76]:
docs_df = pd.DataFrame({"texts":docs})

In [77]:
docs_df

,texts
0,Ad sales boost Time Warner profit\n\nQuarterly...
1,Dollar gains on Greenspan speech\n\nThe dollar...
2,Yukos unit buyer faces loan claim\n\nThe owner...
3,High fuel prices hit BA's profits\n\nBritish A...
4,Pernod takeover talk lifts Domecq\n\nShares in...
5,Japan narrowly escapes recession\n\nJapan's ec...
6,Jobs growth still slow in the US\n\nThe US cre...
7,"India calls for fair trade rules\n\nIndia, whi..."
8,Ethiopia's crop production up 24%\n\nEthiopia ...
9,Court rejects $280bn tobacco case\n\nA US gove...


### Pre-process the docs

In [78]:
import string
import re

def preProcessing(text):
    stemm_snow = SnowballStemmer("english")
    lem = WordNetLemmatizer()
    text = text.replace("\n", " ")
    text = re.sub(r"[0-9]", "", text)
    text = text.lower()
    text = word_tokenize(text)
    text = [stemm_snow.stem(word) for word in text]
    text = [lem.lemmatize(word) for word in text]
    text = [char for char in text if char not in string.punctuation]
    text = [word for word in text if word not in stopwords.words('english')]
    text = ' '.join(text)
    
    return text
    

In [85]:
docs_df['texts'] = docs_df['texts'].apply(preProcessing)

In [90]:
sentences = docs_df['texts']
from nltk.probability import FreqDist
vocabs = []
for sentence in sentences:
    word_dist = FreqDist()
    # update with news words frequency on each iteration
    word_dist.update(sentence.split())
    vocabs.append(word_dist)

In [102]:

len(vocabs)

50

In [104]:
def Top5(sentence):
    match_list = [0]*50
    for word in sentence.split(" "):
        for i in range(len(vocabs)):
            if word in vocabs[i].keys():
                match_list[i]+=1
    top5 = np.array(match_list).argsort()[-5:]    
    
    print("These are the top 5 results based on your search query")
    print("")
    for index in range(4,-1,-1):
        print("================")
        print("Article No: {}".format(5 - index))
        print(docs[top5[index]])


### testing

In [106]:
print("Enter your search query")
search = input()
print()
Top5(search)

Enter your search query
Is studying better than learning and is it the best thing to do in terms of.

These are the top 5 results based on your search query

Article No: 1
Radcliffe tackles marathon tasks

Paula Radcliffe faces arguably the biggest test of her career in the New York City Marathon on Sunday.

Back under the spotlight of public scrutiny she will attempt to erase the double disappointment of the Athens Olympics, where she failed to finish the marathon and then the 10,000m. BBC Sport examines the challenges facing Radcliffe ahead of the big race. 

The ability to run a gruelling 26.2 miles relies largely upon an athlete's belief that they can do it. Every runner will hit the wall at some stage and see written on it, "Are you strong enough to finish?"

The question could hit Radcliffe hard after she was unable to complete her last two races in high-profile and emotional circumstances. Sports psychologist Hugh Richards says the 30-year-old must draw on her past achievements 